# Fuzzy-Lookup

In [1]:
# We import the libraries that we are goin to use. 

import numpy as np
import pandas as pd
from polyfuzz import PolyFuzz
from sys import exit

## Using the Account Name parameter

### Cleansing the SF accounts

In [2]:
# We read the file that contains the SF Accounts.

df2 = pd.read_csv('input/sf_accounts.csv').dropna(subset=['Account ID'])

In [3]:
# We stablish the strange symbols and patters that we are going to remove. 

a = '[^a-z0-9\s]| inc| plc| llc| ltd| llp| limited| group| corporation| uk'

In [4]:
# We create a new Account Name column to clean it.
# We remove those symbols and patters from the Account Name Cleaned column.
# We replace the empty strings with NaN values in the Account Name Cleaned column.

df2['Account Name Cleaned'] = df2['Account Name']
df2['Account Name Cleaned'] = df2['Account Name Cleaned'].str.lower().str.replace(a, '', regex=True).str.strip()
df2['Account Name Cleaned'] = df2['Account Name Cleaned'].replace('', np.nan)

In [5]:
# We create a DataFrame without NaN values in the Account Name Cleaned column. 
# We sort by Account Name Cleaned and Contact Count.
# We drop duplicates in the Account Name Cleaned column and we keep the first one. 

df2_account = df2.dropna(subset=['Account Name Cleaned'])
df2_account = df2_account.sort_values(by=['Account Name Cleaned', 'Contact Count'], ascending=[True, False])
df2_account = df2_account.drop_duplicates(subset='Account Name Cleaned', keep='first')

In [6]:
# We create a list with the Account Name Cleaned values. 

df2_account_list = df2_account['Account Name Cleaned'].tolist()

### Cleansing the external companies

In [7]:
# We read the file that contains the external accounts. 

df3 = pd.read_csv('input/ext_data.csv', encoding="latin-1")


In [8]:
# We create a new Account Name column to clean it.
# We remove those symbols and patters from the Account Name Cleaned column.
# We replace the empty strings with NaN values in the Account Name Cleaned column.

df3['Company Name Cleaned'] = df3['Company Name']
df3['Company Name Cleaned'] = df3['Company Name Cleaned'].str.lower().str.replace(a, '', regex=True).str.strip()
df3['Company Name Cleaned'] = df3['Company Name Cleaned'].fillna(' ')

In [9]:
# We create a list with the Account Name Cleaned values and we drop the duplicates. 

df3_company_list = df3.drop_duplicates(subset='Company Name Cleaned', keep='first')['Company Name Cleaned'].tolist()

In [10]:
# We use the model 'TF-IDF' from the PolyFuzz library.

model = PolyFuzz("TF-IDF")
model.match(df2_account_list, df3_company_list)

In [11]:
# We create a DataFrame that contains similarity > .95

old_account = model.get_matches()[model.get_matches()['Similarity'] > .95].reset_index(drop=True)
old_account = old_account.rename({'From': 'Account Name Cleaned', 'To': 'Company Name Cleaned'}, axis=1)

In [12]:
old_account['T/F'] = old_account['Account Name Cleaned'] == old_account['Company Name Cleaned']

In [13]:
old_account = old_account.sort_values(by=['T/F'], ascending=True).reset_index(drop=True)

In [14]:
old_account.to_csv('output/old_account_check.csv')

In [15]:
exit()

SystemExit: 

C:\Users\PC03\miniconda3\envs\myenv\lib\site-packages\IPython\core\interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [16]:
old_account = pd.read_csv('output/old_account_check.csv')

In [17]:
old_account_T = old_account[old_account['T/F'] == True].reset_index(drop='True')
old_account_F = old_account[old_account['T/F'] == False].reset_index(drop='True')

In [18]:
inner_account_T = pd.merge(df2_account,
                      old_account_T,
                      on = 'Account Name Cleaned',
                      how = 'inner',
                      indicator = True).drop(columns = '_merge')

In [19]:
inner_account_T = inner_account_T[['Account ID', 'Account Name', 'Company Name Cleaned', 'Website']]

In [20]:
inner_company_T = pd.merge(inner_account_T,
                      df3,
                      on = 'Company Name Cleaned',
                      how = 'inner',
                      indicator = True).drop(columns = '_merge')

In [21]:
inner_company_T.to_csv('output/old_accounts.csv')

In [22]:
inner_company_T_list = inner_company_T['Company Name'].tolist()

In [23]:
new_accounts = df3[~df3['Company Name'].isin(inner_company_T_list)].reset_index(drop=True)

In [24]:
new_accounts.to_csv('output/new_accounts.csv')

## Using the Website parameter



### Cleansing the old-new DataFrame